# Convergence plot. Minimal BMN

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
plt.rc("text", usetex=True)
plt.style.use("../figures/paper.mplstyle")

## Read data from disk

In [37]:
# parameters
opt_label = ["COBYLA","L-BFGS-B","SLSQP","NELDER-MEAD"]
var_label = 'ry-rz'
coupling = '02'
depth = '3'
nrep = '10'
maxiter = '10000'

In [38]:
def read_data(optimizer):
    filename = f"../data/miniBMN_l{coupling}_convergence_{optimizer}_{var_label}_depth{depth}_reps{nrep}_max{maxiter}.h5"
    return pd.read_hdf(filename,"vqe")

In [39]:
df = read_data(opt_label[0])

In [40]:
df.head()

,counts,energy
rep,,
0,1,3.565884
0,2,4.808009
0,3,4.211399
0,4,3.881846
0,5,3.836754


In [41]:
best_values = df["energy"].groupby("rep").apply(min)

In [42]:
best_values.sort_values()

rep
2    0.103518
3    0.125041
1    0.150000
0    0.150000
8    0.150000
9    0.150001
6    0.150011
7    0.150021
5    0.150028
4    0.150039
Name: energy, dtype: float64

Collect multiple data in one frame

In [43]:
frames = [read_data(o) for o in opt_label]
result = pd.concat(frames, keys=opt_label,names=['Optimizer'])

In [44]:
result.head()

counts    energy
Optimizer rep                  
COBYLA    0         1  3.565884
          0         2  4.808009
          0         3  4.211399
          0         4  3.881846
          0         5  3.836754

## Get stats

In [45]:
result.groupby('Optimizer').apply(min).energy

Optimizer
COBYLA         0.103518
L-BFGS-B       0.150001
SLSQP          0.150000
NELDER-MEAD    2.910476
Name: energy, dtype: float64

In [46]:
result.loc["COBYLA"].groupby('rep').apply(min).energy

rep
0    0.150000
1    0.150000
2    0.103518
3    0.125041
4    0.150039
5    0.150028
6    0.150011
7    0.150021
8    0.150000
9    0.150001
Name: energy, dtype: float64

In [47]:
for r in opt_label:
    print("Optimizer:",r)
    print("Mean: ",result.loc[r].groupby('rep').apply(min).energy.mean())
    print("Std: ",result.loc[r].groupby('rep').apply(min).energy.std())
    print("Min: ",result.loc[r].groupby('rep').apply(min).energy.min())
    print("Max: ",result.loc[r].groupby('rep').apply(min).energy.max())

Optimizer: COBYLA
Mean:  0.1428658510031972
Std:  0.015897573267420324
Min:  0.10351767672268163
Max:  0.1500390135796044
Optimizer: L-BFGS-B
Mean:  0.15076084938043288
Std:  0.0009209663873685163
Min:  0.15000113711091315
Max:  0.1525410180061702
Optimizer: SLSQP
Mean:  0.15000058887466305
Std:  6.054313843273407e-07
Min:  0.15000012589340295
Max:  0.1500017432120342
Optimizer: NELDER-MEAD
Mean:  3.3906185277924976
Std:  0.32692690492361587
Min:  2.9104759538029032
Max:  3.882205184688683


In [48]:
gs = dict()
for r in opt_label:
    gs[r] = result.loc[r].groupby('rep').apply(min).energy
gsdf = pd.DataFrame.from_dict(gs, dtype=float)

In [49]:
gsdf

,COBYLA,L-BFGS-B,SLSQP,NELDER-MEAD
rep,,,,
0,0.150000,0.151414,0.150002,2.910476
1,0.150000,0.152541,0.150000,3.653528
2,0.103518,0.151003,0.150000,3.587732
3,0.125041,0.150441,0.150001,3.882205
4,0.150039,0.150001,0.150000,3.195625
5,0.150028,0.150110,0.150000,3.565522
6,0.150011,0.150002,0.150001,3.716195
7,0.150021,0.151932,0.150002,3.136449
8,0.150000,0.150144,0.150000,3.183510


In [50]:
gsdf.describe().T[["min","max","mean","std"]]

,min,max,mean,std
COBYLA,0.103518,0.150039,0.142866,1.589757e-02
L-BFGS-B,0.150001,0.152541,0.150761,9.209664e-04
SLSQP,0.150000,0.150002,0.150001,6.054314e-07
NELDER-MEAD,2.910476,3.882205,3.390619,3.269269e-01
